# ロバストMDPと正則化

参考
* [Twice regularized MDPs and the equivalence between robustness and regularization](https://arxiv.org/abs/2110.06267)
* [Distributionally Robust Optimization: A Review](https://arxiv.org/abs/1908.05659): 読もう

[ロバストMDP](RL_robust_MDP.ipynb)とエントロピー正則化には実は密接な関係があります。
実際、この関係は強化学習以外の分野におけるロバスト最適化問題では有名みたいですね ([Distributionally Robust Optimization: A Review](https://arxiv.org/abs/1908.05659)を読もう)。
今回は強化学習におけるロバスト性と正則化がほぼ等価であることを見ていきます。

元論文は報酬関数についてのロバスト性についても議論していますが，ややこしくなるので遷移確率についてのロバスト性だけ解説します．

**強化学習の表記**

* $\mathcal{X}:= \mathcal{S}\times \mathcal{A}$
* 集合$C$について，$\delta(c) = 0$ if $c\in C$ otherwise $\delta(c)=\infty$
* $\mu_0$：初期分布
* 期待リターン：$\rho(\pi):=\mathbb{E}\left[\sum_{t=0}^{\infty} \gamma^t r\left(s_t, a_t\right) \mid \mu_0, \pi, P\right]=\left\langle v_{P}^\pi, \mu_0\right\rangle$
* ベルマン期待作用素：$T_{P}^\pi v:=r^\pi+\gamma P^\pi v \quad \forall v \in \mathbb{R}^{\mathcal{S}}, \pi \in \Delta_{\mathcal{A}}^{\mathcal{S}}$,
* ベルマン最適作用素：$T_{P} v:=\max _{\pi \in \Delta_{\mathcal{A}}^{\mathcal{S}}} T_{P}^\pi v \quad \forall v \in \mathbb{R}^{\mathcal{S}}$
* 貪欲方策：$\mathcal{G}_{P}(v):=\left\{\pi \in \Delta_{\mathcal{A}}^{\mathcal{S}}: T_{P}^\pi v=T_{P} v\right\} \quad \forall v \in \mathbb{R}^{\mathcal{S}}$
* その他：
    * $r^\pi:=\left[\left\langle\pi_s, r(s, \cdot)\right\rangle\right]_{s \in \mathcal{S}}$ 
    * $P^\pi=\left[P^\pi\left(s^{\prime} \mid s\right)\right]_{s^{\prime}, s \in \mathcal{S}}$ 
    * $P^\pi\left(s^{\prime} \mid s\right):=\left\langle\pi_s, P\left(s^{\prime} \mid s, \cdot\right)\right\rangle$

[RL_as_LP.ipynb](RL_as_LP.ipynb)で詳しく解説しましたが、プランニング問題は線形計画問題に変形することができます。
これを使うと、期待リターンを次のように書き換えることができます。

* 主問題：$\rho(\pi) = \min _{v \in \mathbb{R}^{\mathcal{S}}}\left\langle v, \mu_0\right\rangle \text { subject to (s.t.) } v \geq r^\pi+\gamma P^\pi v$
* 双対問題：$\rho(\pi) = \max _{\mu \in \mathbb{R}^{\mathcal{S}}}\left\langle r^\pi, \mu\right\rangle$ s. t. $\mu \geq 0$ and $\left(\mathbf{I d}_{\mathbb{R}^{\mathcal{S}}}-\gamma P_*^\pi\right) \mu=\mu_0$

ここで、$\mathbf{I d}_{\mathbb{R}^{\mathcal{S}}}$は$\mathcal{S}$上の恒等関数、$P_*^\pi$は$P^\pi$の随伴作用素です。つまり、$\left\langle P^\pi v, v^{\prime}\right\rangle=\left\langle v, P_*^\pi v^{\prime}\right\rangle \quad \forall v, v^{\prime} \in \mathbb{R}^{\mathcal{S}}$とします。

この主問題と双対問題の解はそれぞれ価値関数と割引訪問頻度に対応します。詳しくは[RL_as_LP.ipynb](RL_as_LP.ipynb)を見てください。


**正則化**

つづいて、正則化されたMDPを考えていきます。

* $s \in \mathcal{S}, \Omega_s: \Delta_{\mathcal{A}} \rightarrow \mathbb{R}$：強凸関数
* 正則化されたベルマン期待作用素：$\left[T_{P}^{\pi, \Omega} v\right](s):=T_{P}^\pi v(s)-\Omega_s\left(\pi_s\right) \quad \forall v \in \mathbb{R}^{\mathcal{S}}, s \in \mathcal{S}$
* 正則化されたベルマン最適作用素：$T_{P}^{*, \Omega} v:=\max _{\pi \in \Delta_{\mathcal{A}}^{\mathcal{S}}}{T_{P}^{\pi, \Omega}} v \quad \forall v \in \mathbb{R}^{\mathcal{S}}$
* 正則化された価値関数（$T_{P}^{\pi, \Omega}$の不動点）：$v_{P}^{\pi, \Omega}=\left(\mathbf{I}_{\mathcal{S}}-\gamma P^\pi\right)^{-1}\left(r^\pi-\Omega(\pi)\right)$
    * これは$\tilde{r}^\pi:=r^\pi-\Omega(\pi)$のときの通常のMDPと同じ。$r^\pi$がいじられていることに注意。$r$ではないよ。


**ロバストMDP**

最後にロバストMDPの表記です。今回は報酬関数もロバストな場合を考えましょう。

* 不確実集合：$\mathcal{U}\subseteq \Delta_{\mathcal{S}}^{\mathcal{X}}$
    * ロバストMDPでは$P \in \mathcal{U}$の最悪ケースを考えます。
    * 一般の$\mathcal{U}$を考えると解けないことが知られています。そこで状態や状態行動についての直積集合$\mathcal{U}=\times_{s \in \mathcal{S}} \mathcal{U}_s=\times_{s \in \mathcal{S}}$がよく使われます。（$(s, a)$-rectangularも含まれます）
* ロバスト期待ベルマン作用素：${\left[T^{\pi, \mathcal{U}} v\right](s):=\min _{P \in \mathcal{U}} T_{P}^\pi v(s) \quad \forall v \in \mathbb{R}^{\mathcal{S}}, s \in \mathcal{S}, \pi \in \Delta_{\mathcal{S}}^{\mathcal{A}},}$
    * この不動点を$v^{\pi, \mathcal{U}} = \min _{P \in \mathcal{U}} v_{P}^\pi(s)$とします．
* ロバスト最適ベルマン作用素：${\left[T^{*, \mathcal{U}} v\right](s):=\max _{\pi \in \Delta_{\mathcal{A}}^{\mathcal{S}}}\left[T^{\pi, \mathcal{U}} v\right](s) \quad \forall v \in \mathbb{R}^{\mathcal{S}}, s \in \mathcal{S},}$
    * この不動点は$v^{*, \mathcal{U}}(s):=\max _{\pi \in \Delta_{\mathcal{S}}^{\mathcal{A}}} v^{\pi, \mathcal{U}}(s)$です．

ロバスト期待価値は次のように線型計画法でも求まります（[RL_robust_MDP_convex..ipynb](RL_robust_MDP_convex..ipynb)参照）:

$$\max _{v \in \mathbb{R}^{\mathcal{S}}}\left\langle v, \mu_0\right\rangle\; \text{ s.t. }\; v \leq T_{P}^\pi v \;\text{ for all }\; P \in \mathcal{U}\;.$$

## Fenchel-Rockafellar duality

今回の話にはFenchel-Rockafellar dualityが非常に重要になります. 
$X, Y$ をユークリッド空間, $f: X \rightarrow \overline{\mathbb{R}}$ と$g: Y \rightarrow \overline{\mathbb{R}}$ をプロパーな凸関数, $A: X \rightarrow Y$ を$0 \in \operatorname{core}(\operatorname{dom}(g)-$ $A(\operatorname{dom}(f)))$であるような線形写像とします。 
このとき、

$$
\min _{x \in X} f(x)+g(A x)=\max _{y \in Y}-f^*\left(-A^* y\right)-g^*(y) .
$$

が成り立ちます。



## Robust MDP と 正則化

これから先は、真のモデル$P_0$の周りに不確かなMDPが広がってる状況を考えます。
つまり，$\mathcal{U}=P_0 + \mathcal{P}$とします．
このとき，次のようにロバスト期待価値関数は正則化されたベルマン作用素についての凸最適化問題を解くことで求まります．

---

$v^{\pi, \mathcal{U}}$は次の凸最適化問題の最適解になります．

$$
\max _{v \in \mathbb{R}^{\mathcal{S}}}\left\langle v, \mu_0\right\rangle s . t . v(s) \leq T_{P_0}^\pi v(s)-\sigma_{\mathcal{P}_s}\left(-\gamma v \cdot \pi_s\right) \text { for all } s \in \mathcal{S}
$$

ここで，$\left[v \cdot \pi_s\right]\left(s^{\prime}, a\right):=v\left(s^{\prime}\right) \pi_s(a) \quad \forall\left(s^{\prime}, a\right) \in \mathcal{X}$です．

証明しましょう．
まず，これは次のRobust Counterpartに変形できます．　

$$
\max _{v \in \mathbb{R}^{\mathcal{S}}}\left\langle v, \mu_0\right\rangle \text { s. t. } F(s) \leq 0 \text { for all } s \in \mathcal{S} \text {, }
$$
ここで$F(s):=\max _{P \in \mathcal{U}}\left\{v(s)-r^\pi(s)-\gamma P^\pi v(s)\right\}$としました．
この$F(s)$を変形しましょう．

$$
\begin{aligned}
F(s)
& =\max _{P \in \mathcal{P}}\left\{v(s)-r^\pi(s)-\gamma\left(P_0^\pi+P^\pi\right) v(s)\right\} \\
& =\max _{P \in \mathcal{P}}\left\{v(s)-r^\pi(s)-\gamma P_0^\pi v(s)-\gamma P^\pi v(s)\right\} \\
& =\max _{P \in \mathcal{P}}\left\{v(s)-T_{P_0}^\pi v(s)-\gamma P^\pi v(s)\right\} \\
& =\max _{P \in \mathcal{P}}\left\{-\gamma P^\pi v(s)\right\}+v(s)-T_{P_0}^\pi v(s) \\
& =-\min _{P \in \mathcal{P}}\left\{\gamma P^\pi v(s)\right\}+v(s)-T_{P_0}^\pi v(s) \\
& =-\min _{P \in \mathbb{R}^{\mathcal{X} \times \mathcal{S}}}\left\{\gamma P^\pi v(s)+\delta_{\mathcal{P}}(P)\right\}+v(s)-T_{P_0}^\pi v(s) \\
& =-\min _{P \in \mathbb{R}^{\mathcal{X} \times \mathcal{S}}}\left\{\gamma\left\langle P_s^\pi, v\right\rangle+\delta_{\mathcal{P}}(P)\right\}+v(s)-T_{P_0}^\pi v(s) . \\
&
\end{aligned}
$$

ここで，rectangularの仮定より，$P:=\left(P_s\right)_{s \in \mathcal{S}} \in \mathbb{R}^{\mathcal{X}}$とすれば，$\delta_{\mathcal{P}}(P)=\sum_{s^{\prime} \in \mathcal{S}} \delta_{\mathcal{P}_{s^{\prime}}}\left(P_{s^{\prime}}\right)$です．
これを使うと，

$$
\begin{aligned}
\min _{P \in \mathbb{R}^{\mathcal{X} \times \mathcal{S}}}\left\{\gamma\left\langle P_s^\pi, v\right\rangle+\delta_{\mathcal{P}}(P)\right\} & =\min _{P \in \mathbb{R}^{\mathcal{X} \times \mathcal{S}}}\left\{\gamma\left\langle P_s^\pi, v\right\rangle+\sum_{s^{\prime} \in \mathcal{S}} \delta_{\mathcal{P}_{s^{\prime}}}\left(P_{s^{\prime}}\right)\right\} \\
& =\min _{P_s \in \mathbb{R}^{\mathcal{X}}}\left\{\gamma\left\langle P_s^\pi, v\right\rangle+\delta_{\mathcal{P}_s}\left(P_s\right)\right\},
\end{aligned}
$$

が成り立ちます．最初の$\left\langle P_s^\pi, v\right\rangle$は$s$についての話, $\delta_{\mathcal{P}}(P)$は$s, a$についての話なので，それを後半の変形では$s$についての話に変形しています．

よって，

$$
F(s)=-\min _{P_s \in \mathbb{R}^{\mathcal{X}}}\left\{\gamma\left\langle P_s^\pi, v\right\rangle+\delta_{\mathcal{P}_s}\left(P_s\right)\right\}+v(s)-T_{\left(P_0, r_0\right)}^\pi v(s)
$$

が成り立ちます．ここで，$\left[v \cdot \pi_s\right]\left(s^{\prime}, a\right):=v\left(s^{\prime}\right) \pi_s(a)$ for all $\left(s^{\prime}, a\right) \in \mathcal{X}$および$\varphi\left(P_s\right):=\gamma\left\langle P_s^\pi, v\right\rangle$を導入しましょう．これは$\varphi\left(P_s\right)=\gamma\left\langle P_s, v \cdot \pi_s\right\rangle$とも書けます．

このとき，

$$
\min _{P_s \in \mathbb{R}^{\mathcal{X}}}\left\{\gamma\left\langle P_s^\pi, v\right\rangle+\delta_{\mathcal{P}_s}\left(P_s\right)\right\}=\min _{P_s \in \mathbb{R}^{\mathcal{X}}}\left\{\varphi\left(P_s\right)+\delta_{\mathcal{P}_s}\left(P_s\right)\right\}=-\min _{\mathbf{B} \in \mathbb{R}^{\mathcal{X}}}\left\{\varphi^*(-\mathbf{B})+\sigma_{\mathcal{P}_s}(\mathbf{B})\right\}
$$

が成り立ちます．（Fenchel-Rockafellar dualityと$\left(\delta_{\mathcal{P}_s}\right)^*=\sigma_{\mathcal{P}_s}$を使っています．）

さらに，

$$
\begin{aligned}
\varphi^*(-\mathbf{B}) & =\max _{P_s \in \mathbb{R}^\chi}\left\{\left\langle P_s,-\mathbf{B}\right\rangle-\varphi\left(P_s\right)\right\} \\
& =\max _{P_s \in \mathbb{R}^\chi}\left\{\left\langle P_s,-\mathbf{B}\right\rangle-\gamma\left\langle P_s, v \cdot \pi_s\right\rangle\right\} \\
& =\max _{P_s \in \mathbb{R}^\chi}\left\langle P_s,-\mathbf{B}-\gamma v \cdot \pi_s\right\rangle \\
& =\left\{\begin{array}{l}
0 \text { if }-\mathbf{B}-\gamma v \cdot \pi_s=0 \\
+\infty \text { otherwise, }
\end{array}\right.
\end{aligned}
$$

なので，$\min _{\mathbf{B} \in \mathbb{R}^{\mathcal{X}}}\left\{\varphi^*(-\mathbf{B})+\sigma_{\mathcal{P}_s}(\mathbf{B})\right\}=\sigma_{\mathcal{P}_s}\left(-\gamma v \cdot \pi_s\right)$が成り立ちます．よって，

$$
F(s)=\sigma_{\mathcal{P}_s}\left(-\gamma v \cdot \pi_s\right)+v(s)-T_{P_0}^\pi v(s),
$$

です．証明終わり．

---




これは特に$\mathcal{U}_s:=\left\{P_s \in \mathbb{R}^{\mathcal{X}}:\left\|(P_0)_s - P_s\right\| \leq \alpha_s^P\right\}$のとき，もっとキレイに書くことができます．

$$
\max _{v \in \mathbb{R}^{\mathcal{S}}}\left\langle v, \mu_0\right\rangle \text { s. t. } v(s) \leq T_{\left(P_0, r_0\right)}^\pi v(s)-\alpha_s^P \gamma\|v\|\left\|\pi_s\right\| \text { for all } s \in \mathcal{S} \text {. }
$$

こっちの証明は省略します．

